<a href="https://colab.research.google.com/github/YokoSSS/ProgrammingPython/blob/%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96%E6%BC%94%E7%BF%92(20)/%E6%95%B0%E7%90%86%E6%9C%80%E9%81%A9%E5%8C%96%E6%BC%94%E7%BF%92%EF%BC%8820%EF%BC%892024%E5%89%8D%E6%9C%9F4_4_1_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PuLPモジュールとPandasモジュールの読み込み

In [1]:
# Install pulp package
!pip install pulp

import pulp
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 15.0 MB/s eta 0:00:00


## 目的関数の係数を定義
配役の適正度を記入したCSVファイル 4-4-1.csv を読み込む．

In [2]:
df = pd.read_csv('4-4-1.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '4-4-1.csv'

In [ ]:
df['Var'] = [pulp.LpVariable(f'x{df.Member[i]}_{df.Role[i]}',cat=pulp.LpBinary) for i in df.index]
df[:5]

,Member,Role,Fitness,Var
0,1,1,5,x1_1
1,1,2,4,x1_2
2,1,4,3,x1_4
3,2,1,2,x2_1
4,2,4,3,x2_4


## 問題の定義

In [ ]:
p = pulp.LpProblem('割り当て問題', sense=pulp.LpMaximize)
p += pulp.lpDot(df.Fitness, df.Var), '目的関数　配役の評価値'
for i, v in df.groupby('Member'):
    p += pulp.lpSum(v.Var) == 1, f'役員{i}の1役職担当制約'
for j, v in df.groupby('Role'):
    p += pulp.lpSum(v.Var) == 1, f'役職{j}の1人担当制約'
p

割り当て問題:
MAXIMIZE
5*x1_1 + 4*x1_2 + 3*x1_4 + 2*x2_1 + 3*x2_4 + 5*x3_2 + 4*x3_3 + 5*x3_4 + 5*x4_1 + 4*x4_2 + 4*x4_3 + 3*x4_4 + 0
SUBJECT TO
役員1の1役職担当制約: x1_1 + x1_2 + x1_4 = 1

役員2の1役職担当制約: x2_1 + x2_4 = 1

役員3の1役職担当制約: x3_2 + x3_3 + x3_4 = 1

役員4の1役職担当制約: x4_1 + x4_2 + x4_3 + x4_4 = 1

役職1の1人担当制約: x1_1 + x2_1 + x4_1 = 1

役職2の1人担当制約: x1_2 + x3_2 + x4_2 = 1

役職3の1人担当制約: x3_3 + x4_3 = 1

役職4の1人担当制約: x1_4 + x2_4 + x3_4 + x4_4 = 1

VARIABLES
0 <= x1_1 <= 1 Integer
0 <= x1_2 <= 1 Integer
0 <= x1_4 <= 1 Integer
0 <= x2_1 <= 1 Integer
0 <= x2_4 <= 1 Integer
0 <= x3_2 <= 1 Integer
0 <= x3_3 <= 1 Integer
0 <= x3_4 <= 1 Integer
0 <= x4_1 <= 1 Integer
0 <= x4_2 <= 1 Integer
0 <= x4_3 <= 1 Integer
0 <= x4_4 <= 1 Integer

## 最適解の計算と結果の読み取り

In [ ]:
result = p.solve()

In [ ]:
pulp.LpStatus[result]

'Optimal'

In [ ]:
pulp.value(p.objective)

17.0

In [ ]:
df['Val'] = df.Var.apply(pulp.value)
df[df.Val > 0]

,Member,Role,Fitness,Var,Val
0,1,1,5,x1_1,1.0
4,2,4,3,x2_4,1.0
5,3,2,5,x3_2,1.0
10,4,3,4,x4_3,1.0
